# Faster than Python
Sometimes your numerical calculations are too slow. We have already discussed several ways to increase their speed. The simplest one is using numpy and scipy functions instead of python loops.

A second possibility is to use a fortran subroutine converted into a python function with f2py. I've alse mentioned the use of Cython, for those of you familiar with C or its variants.

Today I will introduce a couple of other possibilities: [numba](http://numba.pydata.org/) and [julia](http://julialang.org/).

Numba is a python module that allows "just-in-time" compilation to generate optimized machine code of your functions.

Julia is a completely independent language. Do we need yet another language? Well, Julia is not that independent. Here you have some of its characteristics that render it a close relative to Python:

* Its syntax and its data types are close to Python. It is not object oriented. In my blog, there is [an example](https://ramoncrehuet.wordpress.com/2015/03/10/nufft-with-julia/) of how to translate Python code into Julia
* From julia, you can load Python modules.
* From ipython, you can execute julia code with the `%julia` magici function.
* It integrates perfectly with Jupyter (the language independent notebook), so that you can have an ijulia notebook.

To execute `%julia` magic function you just need to add it to `magics/scripts.py` as explained [here](http://stackoverflow.com/questions/24091373/best-way-to-run-julia-code-in-an-ipython-notebook-or-python-code-in-an-ijulia-n). Of course, you need to install julia first (it comes with the Ubuntu distro).

To have an ijulia notebook, you need to run `Pkg.add("IJulia")` from Julia.

In [3]:
%%julia
println("Hello from Julia. Here is a Julia array")
[1:10]

Hello from Julia. Here is a Julia array
10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10


In [4]:
from numba import jit
from numpy import arange

In [6]:
# jit decorator tells Numba to compile this function.
# The argument types will be inferred by Numba when function is called.
@jit
def sum2d_jit(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result



36.0


In [15]:
a = arange(9999*999).reshape(9999,999)

In [16]:
%timeit sum2d(a)

1 loops, best of 3: 4.88 s per loop


In [17]:
sum2d(a) == sum2d_jit(a)

True

In [ ]:
%timeit sum2d_jit(a)

Let's compare it to the Python sum function and the numpy sum method. As you can see the Python function is slower, because it is coded to work with all types of iterables (lists, arrays, sets, etc). `np.sum` being more specialized can be more efficient. 

Interestingly, our function optimized with numba is almost as fast as the optimized numpy funcion.

In [19]:
%timeit sum(a)
%timeit a.sum()

10 loops, best of 3: 34.6 ms per loop
100 loops, best of 3: 15.9 ms per loop
